In [32]:
# spell correction
import numpy as np
from spellchecker import SpellChecker
%config IPCompleter.greedy=True

spell = SpellChecker()

misspelled = spell.unknown(['something', 'is', 'happennning', 'here'])

for word in misspelled:
    print(spell.correction(word))

happening


In [2]:
# identifying synonyms
from nltk.corpus import wordnet as wn

for syn in wn.synsets('mad'):
    print("Definition: " + syn.definition())
    for l in syn.lemmas():
        print(l.name())
    print('\n')

Definition: roused to anger; - Mark Twain
huffy
mad
sore


Definition: affected with madness or insanity
brainsick
crazy
demented
disturbed
mad
sick
unbalanced
unhinged


Definition: marked by uncontrolled excitement or emotion
delirious
excited
frantic
mad
unrestrained


Definition: very foolish
harebrained
insane
mad




In [3]:
# import data files
import pandas as pd
links = pd.read_csv("data/links.csv")
tags = pd.read_csv("data/tags.csv")
genome_tags = pd.read_csv("data/genome-tags.csv")
genome_scores = pd.read_csv("data/genome-scores.csv")
ratings = pd.read_csv("data/ratings.csv")
cage_movies = pd.read_csv("data/nic-cage.csv")

In [4]:
movies = pd.read_csv("data/movies.csv")
movies.head()

# split the title into two columns: title and year
new = movies["title"].str.split("(", n = 1, expand = True)
movies.drop(columns =["title"], inplace = True)
movies["title"] = new[0]
movies["year"] = new[1].str.replace(')','')
movies.head()

,movieId,genres,title,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995


In [29]:
cage_movies.head()

cage_movies.Movie = cage_movies.Movie.astype(str)
movies.title = movies.title.astype(str)
# try to join tables on movie title (might be different which would really suck) (it's different)
cage_left = pd.merge(cage_movies, movies, left_on = 'Movie', right_on = 'title', how = 'left')
cage_left.head(100)

,Movie,Rating,Character,Voice,Year,RottenTomatoes,movieId,genres,title,year
0,A Score to Settle,NR,Frank Pierce,0,2019,13,NaN,NaN,NaN,NaN
1,Spider-Man: Into the Spider-Verse,PG,Spider-Man Noir,1,2018,97,NaN,NaN,NaN,NaN
2,Between Worlds,R,Joe,0,2018,X,NaN,NaN,NaN,NaN
3,Teen Titans Go! To the Movies,PG,Superman,1,2018,91,NaN,NaN,NaN,NaN
4,211,R,Mike Chandler,0,2018,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
86,Racing with the Moon,PG,Nicky,0,1984,60,NaN,NaN,NaN,NaN
87,Rumble Fish,R,Smokey,0,1983,70,NaN,NaN,NaN,NaN
88,Valley Girl,R,Randy,0,1983,82,NaN,NaN,NaN,NaN
89,Fast Times at Ridgemont High,R,Brad's Bud,0,1982,78,NaN,NaN,NaN,NaN


In [28]:
print(movies.loc[movies['title'] == 'Toy Story'])
movies[movies['title'].str.contains('Toy Story')]

Empty DataFrame
Columns: [movieId, genres, title, year]
Index: []


,movieId,genres,title,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
3021,3114,Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2,1999
14813,78499,Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3,2010
20497,106022,Animation|Children|Comedy,Toy Story of Terror,2013
22633,115875,Adventure|Animation|Children|Comedy|Fantasy,Toy Story Toons: Hawaiian Vacation,2011
22634,115879,Adventure|Animation|Children|Comedy|Fantasy,Toy Story Toons: Small Fry,2011
24062,120468,Animation|Children|Comedy,Toy Story Toons: Partysaurus Rex,2012
24064,120474,Animation|Children,Toy Story That Time Forgot,2014
59767,201588,Adventure|Animation|Children|Comedy,Toy Story 4,2019


In [ ]:
movies.dtypes
cage_movies.dtypes
movies['title']= movies['title'].astype(str) 
print(movies.loc[movies['title'] == 'Valley Girl'])

In [16]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [17]:
tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [8]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [11]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
